In [1]:
import torch
import ultralytics
import cv2
import os
import json
from ultralytics import YOLO  
import val
from val import yolo2coco, coco_eval
from pathlib import Path



In [3]:

# Define your classes here (same order as during YOLO training)
classes = ["pedestrians", "bikes", "bicyclists", "e-scooters", "e-scooterists"]

name_model="yolo11x"
# Load YOLO model
model = YOLO("algos/"+name_model+".pt")

In [9]:

# Define your classes here (same order as during YOLO training)
classes = ["pedestrians", "bikes", "bicyclists", "e-scooters", "e-scooterists"]

name_model="MicroVision-Chalmers-x-20250304"
# Load YOLO model
model = YOLO("algos/"+name_model+".pt")
#model=YOLO('runs/detect/train11/weights/best.pt')
video_name='Test-360'
# Open the video file
video_path = "Videos/" + video_name + ".mp4"
cap = cv2.VideoCapture(video_path)

# Get video frame dimensions and fps
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create output directories if they don't exist
images_dir = "yolo/datasets_"+ video_name +"/one/images"
labels_dir = "yolo/datasets_"+ video_name +"/one/labels"
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Create the classes.txt file
classes_file = "yolo/datasets_"+ video_name +"/one/classes.txt"
with open(classes_file, "w") as class_f:
    for cls in classes:
        class_f.write(f"{cls}\n")

print(f"Classes file created at: {classes_file}")

# Define the codec and create a VideoWriter object
output_path = "predictions/annotated_video_"+ video_name + '_'+name_model +".mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

frame_idx = 0  # Frame counter

# Process video frames
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Run YOLO detection
    results = model.track(frame, persist=True, imgsz=1280, conf=0.3, classes=[0, 1])

    # Visualize results on the frame
    annotated_frame = results[0].plot()

    # Save the current frame as an image for Label Studio
    image_file = os.path.join(images_dir, f"frame_{frame_idx:04d}.jpg")
    cv2.imwrite(image_file, frame)

    # Save detections in YOLO format
    label_file = os.path.join(labels_dir, f"frame_{frame_idx:04d}.txt")
    with open(label_file, "w") as f:
        for r in results[0].boxes:
            cls = int(r.cls)
            x_center, y_center, w, h = r.xywhn[0]  # normalized (x_center, y_center, width, height)

            # Write to YOLO label file
            f.write(f"{cls} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

          
      

    # Write the annotated frame to the output video
    out.write(annotated_frame)

    # Display the annotated frame (optional, can comment out)
    # cv2.imshow("Annotated Frame", annotated_frame)

    # Break loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    frame_idx += 1  # Increment frame counter


# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


Classes file created at: yolo/datasets_Test-360/one/classes.txt

0: 640x1280 10 persons, 882.3ms
Speed: 15.7ms preprocess, 882.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 8 persons, 787.7ms
Speed: 3.3ms preprocess, 787.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 8 persons, 783.7ms
Speed: 3.5ms preprocess, 783.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 7 persons, 761.4ms
Speed: 3.3ms preprocess, 761.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 9 persons, 769.6ms
Speed: 3.1ms preprocess, 769.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 11 persons, 770.0ms
Speed: 3.4ms preprocess, 770.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 9 persons, 779.8ms
Speed: 3.7ms preprocess, 779.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 10 

In [39]:
import yaml

# Définition du dataset
dataset = {
    "train": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/train",
    "val": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/val",
    "nc": 5,  # Nombre de classes
    "names": ['pedestrians', 'bikes', 'bicyclists', 'e-scooterists', 'e-scooters'] # Classes
}

# Sauvegarde dans un fichier
with open("dataset.yaml", "w") as file:
    yaml.dump(dataset, file, default_flow_style=False)

print("Fichier dataset.yaml créé avec succès !")



Fichier dataset.yaml créé avec succès !


## Train

In [4]:
name_model="MicroVision-Chalmers-x-20250304"
# Load YOLO model
model = YOLO("algos/"+name_model+".pt")

In [5]:
results = model.train(data="dataset.yaml", epochs=5, batch=5, imgsz=1280, device='cpu')

Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
engine/trainer: task=detect, mode=train, model=algos/MicroVision-Chalmers-x-20250304.pt, data=dataset.yaml, epochs=5, time=None, patience=100, batch=5, imgsz=1280, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

train: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/train/labels.cache... 34 images, 0 backgrounds, 0 corrupt: 100%|██████████| 34/34 [00:00<?, ?it/s]
val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/val/labels.cache... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]


Plotting labels to runs/detect/train11/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005078125), 173 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1280 train, 1280 val
Using 0 dataloader workers
Logging results to runs/detect/train11
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5         0G      1.118      0.953      1.074         90       1280: 100%|██████████| 7/7 [09:16<00:00, 79.47s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:14<00:00,  7.20s/it]

                   all         12        139      0.898      0.765      0.858      0.728


...

        5/5         0G      1.007     0.6062     0.9661        119       1280: 100%|██████████| 7/7 [07:39<00:00, 65.58s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:14<00:00,  7.02s/it]

                   all         12        139      0.843      0.785       0.86       0.71



5 epochs completed in 0.710 hours.
Optimizer stripped from runs/detect/train11/weights/last.pt, 114.4MB
Optimizer stripped from runs/detect/train11/weights/best.pt, 114.4MB

Validating runs/detect/train11/weights/best.pt...
Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,832,799 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:13<00:00,  6.81s/it]


                   all         12        139        0.9      0.765      0.859      0.731
           pedestrians         12         88       0.91      0.572      0.767      0.627
                 bikes          9         25          1      0.649      0.876      0.705
            bicyclists          8          8      0.978      0.875      0.878      0.841
         e-scooterists          7         15          1      0.729      0.778      0.751
            e-scooters          2          3      0.615          1      0.995      0.733
Speed: 2.2ms preprocess, 1100.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/train11


## Evaluation


In [75]:

# Validate with a custom dataset
name_model="MicroVision-Chalmers-x-distorted-20250318"
# Load YOLO model
model = YOLO("algos/"+name_model+".pt")

metrics = model.val(data="dataset.yaml",conf=0.4) 
print(metrics.box.map)  # map50-95

Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,832,799 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/labels.cache... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:54<00:00, 38.18s/it]


                   all         46        563      0.908      0.636      0.794      0.665
                person         46        420       0.88       0.54      0.733      0.581
               bicycle         23         67      0.927      0.567       0.76      0.629
               cyclist         21         28      0.769      0.714      0.805       0.71
             e-scooter         20         39      0.964      0.692      0.841      0.717
          e-scooterist          8          9          1      0.667      0.833      0.689
Speed: 1.3ms preprocess, 2453.7ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val22
0.6653092581562342


In [42]:
# Définition du dataset
dataset = {
    "train": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_GS010646",
    "val": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie",
    "nc": 2,  # Nombre de classes
    "names": ['persons', 'bikes'] # Classes
}

# Sauvegarde dans un fichier
with open("dataset.yaml", "w") as file:
    yaml.dump(dataset, file, default_flow_style=False)

print("Fichier dataset.yaml créé avec succès !")

name_model="yolo11x"
model=YOLO("algos/"+name_model+".pt")

# Validate with a custom dataset
metrics = model.val(data="dataset.yaml",conf=0.4,imgsz=1280,classes=[0, 1])  
print(metrics.box.map)  # map50-95


Fichier dataset.yaml créé avec succès !
Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,919,424 parameters, 0 gradients, 194.9 GFLOPs


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/labels.cache... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:56<00:00, 38.75s/it]


                   all         46        487      0.699      0.568        0.6      0.465
                person         46        420      0.772      0.614      0.657      0.492
               bicycle         23         67      0.625      0.522      0.544      0.438
Speed: 1.4ms preprocess, 2494.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val19
0.46490512603597517


In [8]:

model_path='/Users/martin.dejaeghere/Microsint2/algos/MicroVision-Chalmers-x-20250304.pt'
yaml_path='/Users/martin.dejaeghere/Microsint2/dataset.yaml'
val_path='/Users/martin.dejaeghere/Microsint2/runs/detect/MicroVision-Chalmers-x-202503045'
data_path='/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one'

In [9]:
coco_eval(model_path, yaml_path, val_path, data_path, 128, 288, split="val", iou=0.45, agnostic_nms=False)

Loading model ...
Run validation ...
Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,832,799 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/val/labels.cache... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:13<00:00, 13.42s/it]


                   all         12        139      0.892      0.726      0.855      0.755
                person         12         88      0.959      0.526      0.762      0.655
               bicycle          9         25      0.953       0.52      0.838      0.697
               cyclist          8          8          1      0.867      0.882      0.866
             e-scooter          7         15          1      0.716      0.801      0.783
          e-scooterist          2          3      0.548          1      0.995      0.777
Speed: 1.7ms preprocess, 1079.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Saving runs/detect/MicroVision-Chalmers-x-202503045/predictions.json...
Results saved to runs/detect/MicroVision-Chalmers-x-202503045
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Acc

In [28]:
from pycocotools.coco import COCO 
from pycocotools.cocoeval import COCOeval


In [37]:
pred_json = f"{val_path}/predictions.json"

    # Convert labels to COCO
annotations_json = f"{val_path}/annotations.json"
yolo2coco(
        yolo_data_dir=Path(data_path, 'val'),
        out_json=annotations_json
    )
        
    # Run COCO evaluation
anno = COCO(annotations_json)
imgIds = sorted(anno.getImgIds())

pred = anno.loadRes(pred_json)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!


In [70]:
val = COCOeval(anno, pred, "bbox")
val.params.imgIds = imgIds
    
    # Update thresholds for object sizes
val.params.areaRng = [
        [0 ** 2, 1e5 ** 2], 
        [0 ** 2, 188 ** 2], 
        [188 ** 2, 288 ** 2], 
        [288 ** 2, 1e5 ** 2]
    ]

val.evaluate()
val.accumulate()
val.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100

In [61]:
import json
import cv2
import matplotlib.pyplot as plt

# 📌 Charger le fichier JSON
json_file = "/Users/martin.dejaeghere/Microsint2/runs/detect/val56/annotations.json"
with open(json_file, "r") as f:
    data = json.load(f)

# 📌 Charger l'image cible
image_name = "frame_0961"
image_path = f"/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/val/images/{image_name}.jpg"

image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"❌ Image '{image_path}' non trouvée. Vérifie le chemin !")

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir en RGB pour Matplotlib

# 📌 Dessiner uniquement les bounding boxes de "frame_0961"
for item in data["annotations"]:
    if item["image_id"] == image_name:  # 🟢 Filtrer uniquement cette image
        x, y, w, h = map(int, item["bbox"])  # Convertir en entier
        print(x)
        print(y)
        print(w)
        print(h)
        category_id = item["category_id"]
        score = item.get("score", None)

        # 📌 Couleur (annotations = bleu, prédictions = vert)
        color = (0, 255, 0) if "score" in item else (255, 0, 0)
        
        # 📌 Dessiner le rectangle
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

        # 📌 Ajouter du texte avec `category_id` et `score`
        label = f"Class {category_id}"
        if score:
            label += f" ({score:.2f})"
        
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# 📌 Afficher l’image avec Matplotlib
plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.axis("off")
plt.show()

# 📌 Sauvegarder l’image avec les boxes (optionnel)
output_path = "output_with_boxes.jpg"
cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
print(f"✅ Image avec bounding boxes sauvegardée sous '{output_path}'")


1224
548
13
43
1102
563
23
35
1002
553
36
74
1645
562
19
47
1682
555
19
40
950
546
9
27
1733
557
10
31


<Figure size 1000x800 with 1 Axes>

✅ Image avec bounding boxes sauvegardée sous 'output_with_boxes.jpg'


In [62]:
import cv2
import matplotlib.pyplot as plt

# Image et dimensions (tu devras ajuster ces valeurs en fonction de ton image réelle)
image_path = "/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/val/images/frame_0961.jpg"
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"❌ Image '{image_path}' non trouvée. Vérifie le chemin !")

image_height, image_width, _ = image.shape  # Récupérer les dimensions de l'image

# Les données en format YOLO (normalisées)
yolo_annotations = [
    "0 0.641272 0.528051 0.007079 0.040538",
    "3 0.580299 0.537974 0.012014 0.032797",
    "0 0.531456 0.546867 0.019154 0.068575",
    "0 0.862417 0.542792 0.010402 0.043799",
    "0 0.881420 0.533255 0.009996 0.037443",
    "0 0.497308 0.518623 0.004964 0.025324",
    "2 0.905942 0.531339 0.005652 0.029572"
]

# Dessiner les boîtes au format YOLO
for annotation in yolo_annotations:
    parts = annotation.split()
    category_id = int(parts[0])
    x_center, y_center, width, height = map(float, parts[1:])

    # Convertir les coordonnées normalisées en pixels
    x_center_pixel = int(x_center * image_width)
    y_center_pixel = int(y_center * image_height)
    width_pixel = int(width * image_width)
    height_pixel = int(height * image_height)

    # Calculer les coordonnées de la boîte : [xmin, ymin, xmax, ymax]
    xmin = x_center_pixel - width_pixel // 2
    ymin = y_center_pixel - height_pixel // 2
    xmax = x_center_pixel + width_pixel // 2
    ymax = y_center_pixel + height_pixel // 2

    # Dessiner le rectangle
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    # Ajouter du texte avec la catégorie
    label = f"Class {category_id}"
    cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Afficher l'image avec les boîtes
plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convertir BGR -> RGB pour Matplotlib
plt.axis("off")
plt.show()

# Sauvegarder l'image avec les boîtes
output_path = "output_with_yolo_boxes.jpg"
cv2.imwrite(output_path, image)
print(f"✅ Image avec bounding boxes YOLO sauvegardée sous '{output_path}'")


<Figure size 1000x800 with 1 Axes>

✅ Image avec bounding boxes YOLO sauvegardée sous 'output_with_yolo_boxes.jpg'


In [58]:
import json
import cv2
import matplotlib.pyplot as plt

# 📌 Chemins des fichiers
json_file = "/Users/martin.dejaeghere/Microsint2/runs/detect/val56/predictions.json"  # Remplace par ton chemin exact
image_path = "/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/val/images/frame_0961.jpg"

# 📌 Charger l'image
image = cv2.imread(image_path)
if image is None:
    raise ValueError(f"⚠️ Erreur : Image non trouvée à {image_path}")

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir en RGB pour Matplotlib

# 📌 Charger le fichier JSON
with open(json_file, "r") as f:
    data = json.load(f)

# 📌 Définition des couleurs par catégorie
category_colors = {
    1: (255, 0, 0),   # Rouge
    2: (0, 255, 0),   # Vert
    3: (0, 0, 255),   # Bleu
    4: (255, 255, 0), # Jaune
    5: (255, 0, 255)  # Magenta
}

# 📌 Tracer les bounding boxes uniquement pour "frame_0961"
for item in data:
    if item["image_id"] == "frame_0961":
        x, y, w, h = map(int, item["bbox"])
        print(x)
        print(y)
        print(w)
        print(h)
        category_id = item["category_id"]
        score = item.get("score", 0)

        # 📌 Sélectionner la couleur selon `category_id`
        color = category_colors.get(category_id, (255, 255, 255))  # Blanc par défaut

        # 📌 Dessiner le rectangle
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

        # 📌 Ajouter du texte (catégorie + score)
        label = f"Class {category_id} ({score:.2f})"
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# 📌 Afficher l’image
plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.axis("off")
plt.show()

# 📌 Sauvegarder l’image
output_path = "frame_0961_labeled.jpg"
cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
print(f"✅ Image avec bounding boxes sauvegardée sous '{output_path}'")


2138
1050
79
139
3592
1054
38
76
3510
1067
43
88
2352
1067
49
66
2612
1040
28
82
3699
1057
23
60
2027
1037
19
50
914
988
20
49
3869
1030
22
46
1382
1006
17
36
3644
1040
18
45
253
997
17
30
207
996
14
46
190
999
19
46
3555
1054
31
76
3877
1028
28
48
220
994
19
45
3543
1062
24
80
2567
1045
11
28
1591
992
20
41
229
996
16
41
1636
1005
16
29
3818
1040
19
84
3667
1046
11
36
2541
1048
10
29
2592
1042
14
35
3402
1068
27
45
3735
1117
20
56
1884
1029
24
70
850
997
12
39
725
999
19
48
1711
1051
26
39
183
1011
23
56
2115
1049
38
44


<Figure size 1000x800 with 1 Axes>

✅ Image avec bounding boxes sauvegardée sous 'frame_0961_labeled.jpg'


In [63]:


# Vérifier si l'image a été correctement chargée
if image is None:
    print(f"❌ Impossible de charger l'image : {image_path}")
else:
    # Obtenir la largeur et la hauteur de l'image
    height, width = image.shape[:2]
    print(f"Hauteur : {height} pixels")
    print(f"Largeur : {width} pixels")


Hauteur : 2048 pixels
Largeur : 4096 pixels
